In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
news_data = pd.read_excel('news.xlsx')

In [55]:
# 카테고리 분포 확인
news_data['category'].value_counts()

# 카테고리 라벨 인코딩
def label_encode(x):
    if x == '사회':
        return 1
    elif x == '경제':
        return 2
    elif x == '정치':
        return 3
    elif x == 'IT/과학':
        return 4
    else:
        return 5

news_data['label'] = news_data['category'].apply(label_encode)

In [ ]:
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# 텍스트 클리닝 함수 정의
def clean_text(text):
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    # 특수문자 및 숫자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # \n 제거
    text = text.replace('\n', '')
    return text

# 텍스트 클리닝 적용
news_data['cleaned_document'] = news_data['document'].apply(clean_text)

# 샘플 데이터 확인
news_data[['document', 'cleaned_document']].head()

,document,cleaned_document
0,■ 방송 : 채널A 뉴스A 라이브 (12시~13시 20분)■ 방송일 : 2023년 ...,방송 채널 뉴스 라이브 시시 분 방송일 년 월 일 수요일 진행 이용환 앵커 ...
1,대통령실은 '순방이 곧 민생'이란 점을 강조해 온 윤석열 대통령이 오늘부터 시작되는...,대통령실은 순방이 곧 민생이란 점을 강조해 온 윤석열 대통령이 오늘부터 시작되는 ...
2,"""메가시티 논의 총선 이후에도 이어질 것""오세훈 서울시장은 15일 서울시청 집무실에...",메가시티 논의 총선 이후에도 이어질 것오세훈 서울시장은 일 서울시청 집무실에서 국민...
3,‘면담’ 조경태 의원도 속도조절론대입 농어촌특례 5~6년 유지 논의‘자치구 아닌 자...,면담 조경태 의원도 속도조절론대입 농어촌특례 년 유지 논의자치구 아닌 자치시 편입 ...
4,"한동훈 법무부 장관의 아내 진은정 변호사가 국무위원 배우자, 주한 외교대사 배우자 ...",한동훈 법무부 장관의 아내 진은정 변호사가 국무위원 배우자 주한 외교대사 배우자 등...


In [36]:
# 토큰화, 불용어 제거, 
okt = Okt()

def tokenizer(raw, pos=["Noun", "Verb", "Adjective"], stopword=["년", "월", "일", "시", "분", "초", "뉴스"]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 
            stem=True    # stemming
            )
            if len(word) > 1 and tag in pos and word not in stopword # 한글자 이상이고, 품사가 명사, 동사, 형용사이고, 불용어가 아닌 토큰만 가져오기
        ]

# 토큰화 적용
news_data['token'] = news_data['document'].apply(tokenizer)   

In [48]:
# 토큰들 하나의 문장으로 이어 붙이기
news_data['tokens'] = [" ".join(doc) for doc in news_data['token']]

0       방송 채널 라이브 방송 수요일 진행 이용환 앵커 출연 강성필 민주당 국민 소통 부위...
1       대통령실 순방 민생 이란 강조 하다 윤석열 대통령 오늘 시작 되다 정상 회의 일정 ...
2       메가시티 논의 총선 이후 이어지다 오세훈 서울시장 서울 시청 집무실 국민 뉴시티 프...
3       면담 조경태 의원 속도 조절 대입 농어촌 특례 유지 논의 자치구 아니다 자치시 편입...
4       한동훈 법무부 장관 아내 은정 변호사 국무위원 배우자 주한 외교 대사 배우자 봉사활...
                              ...                        
1590    한국 서비스 학회 추계 학술 대회 개방 문서 관련 발표 중인 한컴 정지 한글과컴퓨터...
1591    공항 경북 서비스 단계 도입 추진 서울 뉴시스 지난 경상북도청 열리다 드림팀 경상북...
1592    왼쪽 이미애 한국 항공사 부사 성철 한화 시스템 대표이사 이철우 경상북도지사 이종호...
1593    이다 수능 호송 지원 올해 맞다 쉴더스 학년 대학 수학 능력 시험 당일 수험생 신속...
1594    디지털 일리 백지영 기자 텔레콤 학년 대학 수학 능력 시험 수험생 다양하다 혜택 제...
Name: tokens, Length: 1595, dtype: object

In [51]:
# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer()

# 텍스트 벡터화
tfidf_matrix = tfidf_vectorizer.fit_transform(news_data['tokens'])

# TF-IDF 벡터화 결과 확인 (feature names 및 matrix shape)
feature_names = tfidf_vectorizer.get_feature_names()
tfidf_matrix_shape = tfidf_matrix.shape

feature_names[:10], tfidf_matrix_shape # 처음 10개의 feature 이름과 행렬의 크기를 출력

(['가가', '가게', '가격', '가격표', '가결', '가계', '가계부채', '가공', '가공업', '가교'],
 (1595, 13873))

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score

In [65]:
# X, y 분리
X = tfidf_matrix.toarray()
y = news_data['label']

# train, test로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
# 가우시안 나이브 베이즈 모델 학습
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [67]:
# 학습한 모델을 통해 test data 예측하기
y_pred = model.predict(X_test)

In [70]:
# 예측 성능 평가
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.99      0.97      0.98        93
           2       0.99      1.00      0.99        87
           3       1.00      0.99      0.99        92
           4       1.00      1.00      1.00        27
           5       0.86      0.95      0.90        20

    accuracy                           0.98       319
   macro avg       0.97      0.98      0.97       319
weighted avg       0.99      0.98      0.98       319

